In [72]:
# 라이브러리
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as spst

# 한글폰트 지정(맑은고딕)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 고해상도
%config InlineBackend.figure_format = 'retina'

In [73]:
data= pd.read_csv('data_1.csv')
data =data.drop(columns ='Unnamed: 0', axis =1 )
data.head()

,날짜시간,추석기준,연휴길이,서울강수량,부산강수량,소요시간(분),통행량
0,2018-09-22 00:00:00,D-2,5,0.0,0.0,289,80601.0
1,2018-09-22 01:00:00,D-2,5,0.0,0.0,286,58863.0
2,2018-09-22 02:00:00,D-2,5,0.0,0.0,287,47431.0
3,2018-09-22 03:00:00,D-2,5,0.0,0.0,285,45375.0
4,2018-09-22 04:00:00,D-2,5,0.0,0.0,323,55734.0


In [85]:
data['날짜시간'] = pd.to_datetime(data['날짜시간'].str.slice(0,13) ,format= '%Y-%m-%d %H')

In [86]:
data1 = data.copy()

In [146]:
data1['추석기준'].value_counts()

D-2    96
D-1    96
Name: 추석기준, dtype: int64

In [163]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LinearRegression
from sklearn.metrics import mean_absolute_error ,r2_score

target = '소요시간(분)'

# 날짜시간 드랍, 추석기준 D삭제
# data = data.drop(columns = '날짜시간', axis =1)
data1 = data1.loc[(data1['추석기준']!='D'),:]

# x,y 셋 분리
x = data1.drop(target , axis = 1 )
y = data1[target]

# 추석기준 가변수화 후 노말라이즈
x = pd.get_dummies(data = x)
x = ( x - x.min() )/( x.max() - x.min() ) 

# 추석연휴
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.1, shuffle=True)

# 선형 회귀 채택
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# 성능 평가
print('MAE: ',mean_absolute_error(y_test, y_pred))
print('R2 : ',r2_score(y_test,y_pred))

result = pd.DataFrame(y_test)
result.columns = ['Actual']
result['Predict'] = y_pred.astype(int)
result['diff'] = abs(result['Actual'] - result['Predict'])

result.head()

MAE:  30.933264565592104
R2 :  0.4296214117307515


,Actual,Predict,diff
194,264,237,27
214,381,257,124
152,287,310,23
216,228,267,39
144,285,291,6


In [164]:
print(list(x))
print(model.coef_)

['날짜시간', '연휴길이', '서울강수량', '부산강수량', '통행량', '추석기준_D-1', '추석기준_D-2']
[-52.93087067 -64.22466135 -40.39730792 -64.8472493   68.82154057
  12.60410279 -12.60410279]


In [165]:
from sklearn.model_selection import cross_val_score

cv_result = cross_val_score(model, x_train, y_train, cv = 10)

# print(cv_result)
print(cv_result.mean())

-0.30462653298571407


# 2. 0~3시부터 20 ~ 23 삭제 해보자

In [122]:
data1.head()

,날짜시간,추석기준,연휴길이,서울강수량,부산강수량,소요시간(분),통행량
0,2018-09-22 00:00:00,D-2,5,0.0,0.0,289,80601.0
1,2018-09-22 01:00:00,D-2,5,0.0,0.0,286,58863.0
2,2018-09-22 02:00:00,D-2,5,0.0,0.0,287,47431.0
3,2018-09-22 03:00:00,D-2,5,0.0,0.0,285,45375.0
4,2018-09-22 04:00:00,D-2,5,0.0,0.0,323,55734.0


In [123]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 239
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   날짜시간     192 non-null    datetime64[ns]
 1   추석기준     192 non-null    object        
 2   연휴길이     192 non-null    int64         
 3   서울강수량    192 non-null    float64       
 4   부산강수량    192 non-null    float64       
 5   소요시간(분)  192 non-null    int64         
 6   통행량      192 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 12.0+ KB


In [166]:
data2 = data1.loc[ (data1['날짜시간'].dt.hour >3) & (data1['날짜시간'].dt.hour < 21) , :]
data2.head()

,날짜시간,추석기준,연휴길이,서울강수량,부산강수량,소요시간(분),통행량
4,2018-09-22 04:00:00,D-2,5,0.0,0.0,323,55734.0
5,2018-09-22 05:00:00,D-2,5,6.5,0.0,282,88672.0
6,2018-09-22 06:00:00,D-2,5,0.0,0.0,329,140577.0
7,2018-09-22 07:00:00,D-2,5,0.0,0.0,338,174733.0
8,2018-09-22 08:00:00,D-2,5,0.0,0.0,331,202078.0


In [167]:
# 날짜시간 드랍, 추석기준 D삭제
data2 = data2.drop(columns = '날짜시간', axis =1)
data2 = data2.loc[(data2['추석기준']!='D'),:]


In [180]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LinearRegression
from sklearn.metrics import mean_absolute_error ,r2_score

target = '소요시간(분)'

# x,y 셋 분리
x = data2.drop(target , axis = 1 )
y = data2[target]

# 추석기준 가변수화 후 노말라이즈
x = pd.get_dummies(data = x , drop_first = True)
x = ( x - x.min() )/( x.max() - x.min() ) 

# 추석연휴
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.1, shuffle=True)

# 선형 회귀 채택
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# 성능 평가
print('MAE: ',mean_absolute_error(y_test, y_pred))
print('R2 : ',r2_score(y_test,y_pred))

result = pd.DataFrame(y_test)
result.columns = ['Actual']
result['Predict'] = y_pred.astype(int)
result['diff'] = abs(result['Actual'] - result['Predict'])

result.head()

MAE:  43.80718061482805
R2 :  -0.7329135729460385


,Actual,Predict,diff
236,264,318,54
231,297,343,46
28,316,297,19
91,360,397,37
37,391,340,51


In [184]:
print(list(x))
print(model.coef_)
print(model.coef_ * x)

['연휴길이', '서울강수량', '부산강수량', '통행량', '추석기준_D-2']
[-93.0116824  -34.48513578 -85.25634361  54.76056876 -31.84219821]
          연휴길이    서울강수량  부산강수량        통행량   추석기준_D-2
4   -93.011682 -0.00000   -0.0   4.304881 -31.842198
5   -93.011682 -7.59842   -0.0   9.992573 -31.842198
6   -93.011682 -0.00000   -0.0  18.955462 -31.842198
7   -93.011682 -0.00000   -0.0  24.853477 -31.842198
8   -93.011682 -0.00000   -0.0  29.575376 -31.842198
..         ...      ...    ...        ...        ...
232 -93.011682 -0.00000   -0.0  46.910939  -0.000000
233 -93.011682 -0.00000   -0.0  43.107518  -0.000000
234 -93.011682 -0.00000   -0.0  35.471146  -0.000000
235 -93.011682 -0.00000   -0.0  28.602159  -0.000000
236 -93.011682 -0.00000   -0.0  24.454589  -0.000000

[136 rows x 5 columns]


# 3. 0~3시부터 20 ~ 23 삭제 해보자  + 2022를 예측해보자

In [139]:
data_2022 = pd.read_excel('2022.xlsx')
# data_2022.
data_2022.tail()

,시간,추석기준,소요시간(분),연휴길이,연도
139,19,D+2,NaN,4,2022
140,20,D+2,NaN,4,2022
141,21,D+2,NaN,4,2022
142,22,D+2,NaN,4,2022
143,23,D+2,NaN,4,2022


# 4.결론 채택 feature 
- 19 년도 교통량,연휴길이,서울강수량,부산강수량,추석기준 D-2, D-1
- 10시 ~ 14시 데이터만 사용

- target = '소요시간(분)'

In [173]:
# D-2,D-1 만있는데이터
data1

,날짜시간,추석기준,연휴길이,서울강수량,부산강수량,소요시간(분),통행량
0,2018-09-22 00:00:00,D-2,5,0.0,0.0,289,80601.0
1,2018-09-22 01:00:00,D-2,5,0.0,0.0,286,58863.0
2,2018-09-22 02:00:00,D-2,5,0.0,0.0,287,47431.0
3,2018-09-22 03:00:00,D-2,5,0.0,0.0,285,45375.0
4,2018-09-22 04:00:00,D-2,5,0.0,0.0,323,55734.0
...,...,...,...,...,...,...,...
235,2021-09-20 19:00:00,D-1,5,0.0,0.0,274,196442.0
236,2021-09-20 20:00:00,D-1,5,0.0,0.0,264,172423.0
237,2021-09-20 21:00:00,D-1,5,0.0,0.0,264,150673.0
238,2021-09-20 22:00:00,D-1,5,0.0,0.0,274,113306.0


In [249]:
data_2019 = data1.loc[(data['날짜시간'].dt.year == 2019),:]
data_2019 = data_2019.loc[ (data_2019['날짜시간'].dt.hour >4) & (data_2019['날짜시간'].dt.hour < 19) , :]
data_2019.reset_index(drop=True , inplace = True)
data_2019['시간']=data_2019['날짜시간'].dt.hour
data_2019['시간'] = data_2019['시간'].astype(str)
tmp = data_2019.set_index('날짜시간' ,drop = True)
tmp

,추석기준,연휴길이,서울강수량,부산강수량,소요시간(분),통행량,시간
날짜시간,,,,,,,
2019-09-11 05:00:00,D-2,4,0.1,8.3,269,76491.0,5
2019-09-11 06:00:00,D-2,4,0.0,0.1,280,158673.0,6
2019-09-11 07:00:00,D-2,4,0.1,0.5,279,261179.0,7
2019-09-11 08:00:00,D-2,4,0.0,0.0,304,287695.0,8
2019-09-11 09:00:00,D-2,4,0.0,0.0,289,249839.0,9
2019-09-11 10:00:00,D-2,4,0.0,0.0,299,256038.0,10
2019-09-11 11:00:00,D-2,4,0.0,0.0,316,261152.0,11
2019-09-11 12:00:00,D-2,4,0.0,0.0,339,258552.0,12
2019-09-11 13:00:00,D-2,4,0.0,0.0,347,273899.0,13


In [250]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28 entries, 2019-09-11 05:00:00 to 2019-09-12 18:00:00
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   추석기준     28 non-null     object 
 1   연휴길이     28 non-null     int64  
 2   서울강수량    28 non-null     float64
 3   부산강수량    28 non-null     float64
 4   소요시간(분)  28 non-null     int64  
 5   통행량      28 non-null     float64
 6   시간       28 non-null     object 
dtypes: float64(3), int64(2), object(2)
memory usage: 1.8+ KB


In [251]:
tmp = tmp.drop(columns = ['연휴길이','서울강수량','부산강수량'], axis =1)

In [259]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LinearRegression
from sklearn.metrics import mean_absolute_error ,r2_score

target = '소요시간(분)'

# 날짜 시간 드랍
# tmp = tmp.drop(columns = ['연휴길이','서울강수량','부산강수량'], axis =1)
# x,y 셋 분리
x = tmp.drop(target , axis = 1 )
y = tmp[target]

# 추석기준 가변수화 후 노말라이즈
x = pd.get_dummies(data = x , drop_first = True)
x = ( x - x.min() )/( x.max() - x.min() ) 

for i in range(10):
    # 추석연휴
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,shuffle=True)

    # 선형 회귀 채택
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    # 성능 평가
    print('MAE: ',mean_absolute_error(y_test, y_pred))
    print('R2 : ',r2_score(y_test,y_pred))

    result = pd.DataFrame(y_test)
    result.columns = ['Actual']
    result['Predict'] = y_pred.astype(int)
    result['diff'] = abs(result['Actual'] - result['Predict'])

    print(result.head())
    print('-------------------------------------------')

MAE:  35.55563406744333
R2 :  0.8330207902692675
                     Actual  Predict  diff
날짜시간                                      
2019-09-11 07:00:00     279      311    32
2019-09-12 14:00:00     498      477    21
2019-09-11 12:00:00     339      289    50
2019-09-11 14:00:00     357      304    53
2019-09-12 15:00:00     511      544    33
-------------------------------------------
MAE:  24.14650144215535
R2 :  0.8278003197173713
                     Actual  Predict  diff
날짜시간                                      
2019-09-11 12:00:00     339      309    30
2019-09-11 11:00:00     316      339    23
2019-09-11 18:00:00     346      320    26
2019-09-11 14:00:00     357      335    22
2019-09-12 09:00:00     470      451    19
-------------------------------------------
MAE:  38.26656016930565
R2 :  0.683076990441009
                     Actual  Predict  diff
날짜시간                                      
2019-09-12 07:00:00     468      439    29
2019-09-12 05:00:00     481      41